<a href="https://colab.research.google.com/github/Alebraco/phagetool/blob/main/Phage_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bio
from Bio import Entrez,SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.3 MB/s eta 0:00:00


In [3]:
Entrez.email = 'alekey039@hotmail.com'

In [4]:
handle = Entrez.einfo(db='ipg')
info = Entrez.read(handle)
print(info)

{'DbInfo': {'DbName': 'ipg', 'MenuName': 'Identical Protein Groups', 'Description': 'Identical Protein Groups DB', 'DbBuild': 'Build230906-1523.1', 'Count': '635926916', 'LastUpdate': '2023/09/10 17:05', 'FieldList': [{'Name': 'ALL', 'FullName': 'All Fields', 'Description': 'All terms from all searchable fields', 'TermCount': '12142932008', 'IsDate': 'N', 'IsNumerical': 'N', 'SingleToken': 'N', 'Hierarchy': 'N', 'IsHidden': 'N'}, {'Name': 'UID', 'FullName': 'UID', 'Description': 'Unique number assigned to each sequence', 'TermCount': '0', 'IsDate': 'N', 'IsNumerical': 'Y', 'SingleToken': 'Y', 'Hierarchy': 'N', 'IsHidden': 'Y'}, {'Name': 'FILT', 'FullName': 'Filter', 'Description': 'Limits the records', 'TermCount': '44', 'IsDate': 'N', 'IsNumerical': 'N', 'SingleToken': 'Y', 'Hierarchy': 'N', 'IsHidden': 'N'}, {'Name': 'WORD', 'FullName': 'Text Word', 'Description': 'Free text associated with record', 'TermCount': '9378921', 'IsDate': 'N', 'IsNumerical': 'N', 'SingleToken': 'N', 'Hiera

In [6]:
def retrieve_ids(max, query):
  ids = []
  start = 0
  while(True):
    handle = Entrez.esearch(db='ipg', retmax = max, retstart = start, term = query)
    rec = Entrez.read(handle)
    handle.close()

    if len(rec['IdList']) == 0:
      break

    start += max
    ids = ids + rec['IdList']
  return ids


In [7]:
max = 40
strain = 'Escherichia coli K-12'
query = str(strain)+'[ORGN] AND receptor[All fields]'
print(query)
#Execute the function with these parameters

ids = retrieve_ids(max, query)
print(ids)

Escherichia coli K-12[ORGN] AND receptor[All fields]
['99311677', '82688357', '20936421', '20841363', '20535002', '20529826', '19855304', '19617331', '19134810', '19134661', '19134027', '19133755', '19133590', '19133548', '17137494', '16942840', '16178699', '16172574', '16168576', '15744767', '15116154', '14936153', '14591894', '14479418', '13743353', '6167027', '5862297', '5547282', '5547039', '5177685', '4913089', '4730850', '4730746', '4730553', '4728879', '4727165', '4715503', '4649108', '4647312', '1891995', '1709242', '1426875', '1296017', '1208676', '1187459', '1154499', '1119582', '976090', '915499', '858224', '820819', '816704', '811254', '789954', '744960', '709215', '701992', '640980', '593232', '587940', '573802', '563601', '345673', '309026', '267293', '253245', '245239', '220841', '201679', '131912', '85293', '4794', '2565', '110357624', '99312242', '71486710', '47623635', '43183145', '43183059', '43183043', '43182622', '43182559', '43181650', '43177438', '41907683', '419

In [8]:
def retrieve_summary(ids, max):
  titles = []
  acc = []
  start = 0
  while start < len(ids):
    idsfrag = ids[start:start + max]
    handle = Entrez.esummary(db='ipg', id = idsfrag, retmax = max)
    ipgsum = Entrez.read(handle)
    handle.close()

    for entry in ipgsum['DocumentSummarySet']['DocumentSummary']:
      titles.append(entry['Title'])
      acc.append(entry['Accession'])

    start += max
  return titles, acc

In [9]:
max = 40
#Execute the function with these parameters
titles, acc = retrieve_summary(ids, max)

In [10]:
handle = Entrez.esummary(db="ipg", id = ids)
ipgsum = Entrez.read(handle)
handle.close()
titles = [entry['Title'] for entry in ipgsum['DocumentSummarySet']['DocumentSummary']]
acc = [entry['Accession'] for entry in ipgsum['DocumentSummarySet']['DocumentSummary']]


In [15]:
#Retrieving AA sequences
handle = Entrez.efetch(db = 'protein', id = acc, rettype = 'gb', retmode = 'text')
output = list(SeqIO.parse(handle, 'gb'))
handle.close()

In [16]:
#Reading sequences and adding them to a list
aaseqs = []
aaseqs = [str(entry.seq) for entry in output]

['...', 'M', 'MLLAVFDRAALMLICLFFLIRIRLFRELLHKSAHSPKELLAVTAIFSLFALFSTWSGVPVEGSLVNVRIIAVMSGGILFGPWVGIITGVIAGIHRYLIDIGGVTAIPCFITSILAGCISGWINLKIPKAQRWRVGILGGMLCETLTMILVIVWAPTTALGIDIVSKIGIPMILGSVCIGFIVLLVQSVEGEKEASAARQAKLALDIANKTLPLFRHVNSESLRKVCEIIRDDIHADAVAITNTDHVLAYVGVGEHNYQNGDDFISPTTRQAMNYGKIIIKNNDEAHRTPEIHSMLVIPLWEKGVVTGTLKIYYCHAHQITSSLQEMAVGLSQIISTQLEVSRAEQLREMANKAELRALQSKINPHFLFNALNAISSSIRLNPDTARQLIFNLSRYLRYNIELKDDEQIDIKKELYQIKDYIAIEQARFGDKLTVIYDIDEEVNCCIPSLLIQPLVENAIVHGIQPCKGKGVVTISVAECGNRVRIAVRDTGHGIDPKVIERVEANEMPGNKIGLLNVHHRVKLLYGEGLHIRRLEPGTEIAFYIPNQRTPVASQATLLL', 'MKMTRLYPLALGGLLLPAIANAQTSQQDESTLVVTASKQSSRSASANNVSSTVVSAPELSDAGVTASDKLPRVLPGLNIENSGNMLYSTISLRGVSSAQDFYNPAVTLYVDGVPQLSTNTIQALTDVQSVELLRGPQGTLYGKSAQGGIINIVTQQPDSTPRGYIEGGVSSRDSYRSKFNLSGPIQDGLLYGSVTLLRQVDDGDMINPATGSDDLGGTRASIGNVKLRLAPDDQPWEMGFAASRECTRATQDAYVGWNDIKGRKLSISDGSPDPYMRRCTDSQTLSGKYTTDDWVFNLISAWQQQHYSRTFPSGSLIVNMPQRWNQDVQELRAATLGDARTVDMVFGLYRQNTREKLNSAYDMPTMPYLSSTGYTTAETLAAYSDLTWHLTDRFDIGGGVRFSHDKSSTQYHGSMLGNPFGDQ

In [18]:
#Naming unnamed proteins
c = 1
for i in range(len(titles)):
  if titles[i] == '':
    titles[i] = 'unnamed protein v'+str(c)
    c += 1


In [21]:
#Version 1 dictionary
#If multiple AA sequences for a protein, add them to a unique key
#Unique keys, multiple values

protein_dictionary = {}
for key, value in zip(titles, aaseqs):
  if key not in protein_dictionary:
    protein_dictionary[key] = [value]
  else:
    protein_dictionary[key].append(value)


In [23]:
#Version 2 dictionary
#Create an alternative list that accounts for same protein names
#Unique keys, unique values

titles_version = []
key_counter = {}

for key in titles:
  if key in key_counter.keys():
    key_counter[key] += 1
    key_title = str(key) + ' v' + str(key_counter[key])
  else:
    key_counter[key] = 1
    key_title = key
  titles_version.append(key_title)


In [96]:
#Creating DataFrame of Version 1 dictionary
df = pd.DataFrame(list(protein_dictionary.keys()), columns=['Protein Name'])
df[strain] = df['Protein Name'].isin(titles)

df

,Protein Name,Escherichia coli K-12
0,TonB-dependent receptor plug domain-containing...,True
1,MFS transporter family glucose-6-phosphate rec...,True
2,ferric-rhodotorulic acid/ferric-coprogen recep...,True
3,ferric aerobactin receptor IutA,True
4,long-chain fatty acid transporter FadL,True
...,...,...
74,Hypothetical protein,True
75,tonB-dependent receptor yncD,True
76,ferrienterobactin receptor precursor,True
77,phage lambda receptor protein,True


In [88]:
#Creating DataFrame of Version 2 dictionary
#Compared dictionary values (accumulative) with AA sequence values (unique for each query)
#Returns True/False

df2 = pd.DataFrame(list(protein_dictionary_v2.items()), columns=['Protein Name','AAseq'])
df2[strain] = df2['AAseq'].isin(aaseqs)
df2.drop('AAseq', axis = 1, inplace = True)

df2

,Protein Name,Escherichia coli K-12
0,TonB-dependent receptor plug domain-containing...,True
1,TonB-dependent receptor plug domain-containing...,True
2,MFS transporter family glucose-6-phosphate rec...,True
3,TonB-dependent receptor plug domain-containing...,True
4,ferric-rhodotorulic acid/ferric-coprogen recep...,True
...,...,...
365,TonB-dependent receptor plug domain-containing...,True
366,TonB-dependent receptor v23,True
367,ferrienterobactin receptor precursor,True
368,phage lambda receptor protein,True
